In [1]:
import xarray as xr
import sys
import pandas as pd
import numpy as np
import xesmf as xe
import warnings
import dask
import sys

from CASutils import calendar_utils as cal

warnings.filterwarnings('ignore')

In [2]:
import dask
dask.config.set({'distributed.dashboard.link': 'https://jupyterhub.ucar.edu/dav/user/{USER}/proxy/{port}/status'})

In [3]:
#from dask_jobqueue import PBSCluster
#from dask.distributed import Client
#cluster = PBSCluster(cores=144, processes=144, memory='450GB', 
#                      resource_spec ='select=4:ncpus=36:mem=450GB', 
#                      walltime='02:00:00', queue='regular', 
#                      project='P04010022')
#client = Client(cluster)

In [16]:
# get the workers going
ncores = 1
nworkers = 24
nmems="25GB"
from dask.distributed import Client
from dask_jobqueue import SLURMCluster
cluster = SLURMCluster(
cores = ncores, processes = ncores, project="P04010022", walltime="04:00:00")
cluster.scale(nworkers)
client = Client(cluster)

In [12]:
# get the workers going
#ncores = 36
#nmem = '460GB'
#nmem = str(int(365*ncores/36))+'GB'
#from dask_jobqueue import SLURMCluster
#from dask.distributed import Client
#cluster = SLURMCluster(cores=ncores,
#                     processes=ncores, memory=nmem,
#                     project='P04010022',
#                     walltime='12:00:00')
#cluster.scale(ncores)
#client = Client(cluster)

In [17]:
cluster

SLURMCluster('tcp://10.12.205.27:33582', workers=0, threads=0, memory=0 B)

In [15]:
#cluster.close()

distributed.client - ERROR - Failed to reconnect to scheduler after 10.00 seconds, closing client
_GatheringFuture exception was never retrieved
future: <_GatheringFuture finished exception=CancelledError()>
concurrent.futures._base.CancelledError


In [18]:
# do this until you see you've got some workers
client

Client Scheduler: tcp://10.12.205.27:33582 Dashboard: https://jupyterhub.ucar.edu/dav/user/islas/proxy/39864/status,Cluster Workers: 21 Cores: 21 Memory: 525.00 GB


In [19]:
# location of ERA5 data on RDA
filepath="/gpfs/fs1/collections/rda/data/ds633.0/e5.oper.an.pl/"
# output location
outpath="/glade/scratch/islas/processed/era5/TEMdiags/"

In [20]:
ystart=1993 ; yend=1993 ; nyears=yend-ystart+1

In [21]:
# open up CESM data to get the output grid.
cesmdat = xr.open_dataset("/glade/campaign/cesm/collections/cesmLE/CESM-CAM5-BGC-LE/atm/proc/tseries/monthly/PHIS/f.e11.F1850C5CNTVSST.f09_f09.002.cam.h0.PHIS.040101-050012.nc")
grid_out = xr.Dataset({'lat': (['lat'], cesmdat.lat)}, {'lon': (['lon'], cesmdat.lon)})

In [22]:
%%time
reusewgt=True
wgtfile=outpath+"wgtfile.nc"
for iyear in range(ystart,yend+1,1):
    #for imon in range(1,13,1):
    for imon in range(12,13,1):
        monstr=str(imon).zfill(2)
        print(str(iyear)+"-"+monstr)
        
        ds = xr.open_mfdataset(
        filepath + str(iyear)+monstr+ "/*_[u, v, w, t].*.nc",
        coords="minimal",
        join="override",
        decode_times=True,
        use_cftime=True,
        chunks={"time":24, "level":15},
        parallel=True,
        data_vars = "minimal",
        compat = "override")
        
        ds = ds.set_coords("utc_date")
        
        dayspermon = ds.time.dt.daysinmonth.data
        dayendstr = str(dayspermon[0])
        timeout = pd.date_range(
            start=str(iyear)+"-"+monstr+"-01",
            end=str(iyear)+"-"+monstr+"-"+dayendstr)
        
        ds = ds.loc[{"time": ds.time.dt.hour.isin([0,6,12,18])}]
             
        ds["T"] = ds.T * (ds.T.level / 1000.0)**(-2./7.)
              
        regridder = xe.Regridder(ds.U, grid_out, 'bilinear', periodic=True, 
                                 reuse_weights=reusewgt, filename=wgtfile)
        reusewgt = True
        
        regridded = regridder(ds).persist()
        
        # this was causing issues before.  Either need to compute the zonal mean before
        # subtracting or do the persist on the regridded above.
        zonal_means = regridded.mean("lon")
        anomaly = regridded - zonal_means
        
        anomaly['uv'] = anomaly.U*anomaly.V
        anomaly['vt'] = anomaly.V*anomaly.T
        anomaly['uw'] = anomaly.U*anomaly.W
        
        zonal_means = zonal_means.merge(anomaly[['uv','vt','uw']].mean("lon"))
        
        temdiags = zonal_means.rename(
            {
                "uv":"UVzm",
                "vt":"VTHzm",
                "uw":"UWzm",
                "U":"Uzm",
                "V":"Vzm",
                "W":"Wzm",
                "T":"THzm",
            })

        temdiags = temdiags.groupby('time.dayofyear').mean()
        temdiags = temdiags.rename({'dayofyear':'time'})
        temdiags['time'] == timeout
        outfile = outpath+"fluxes_"+str(iyear)+"-"+monstr+".nc"
        # Since temdiags is small, it's better to load it and write in serial than in
        # parallel to netcdf.
        temdiags.load().to_netcdf(path=outfile)
        
        client.cancel(regridded) # deleted from distriubted RAM to recover memory
        ds.close()


1993-12
using dimensions ('latitude', 'longitude') from data variable T as the horizontal dimensions for this dataset.
CPU times: user 21.6 s, sys: 12.5 s, total: 34 s
Wall time: 7min 23s
